
# High-level PyTorch or 'What is `torch.nn` *really*'?

Disclaimer: This notebook is an adopted version of [this tutorial](https://pytorch.org/tutorials/beginner/nn_tutorial.html) by Jeremy Howard.

PyTorch provides the elegantly designed modules and classes [torch.nn](https://pytorch.org/docs/stable/nn.html>),
[torch.optim](https://pytorch.org/docs/stable/optim.html), [Dataset](https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset>) and [DataLoader](https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader) to help you create and train neural networks. In order to fully utilize their power and customize them for your problem, you need to really understand exactly what they're doing. To develop this understanding, we will first train basic neural net on the MNIST data set without using any features from these models; we will initially only use the most basic PyTorch tensor functionality. Then, we will incrementally add one feature from ``torch.nn``, ``torch.optim``, ``Dataset``, or ``DataLoader`` at a time, showing exactly what each piece does, and how it works to make the code either more concise, or more flexible.

**Note:** this tutorial assumes you already have PyTorch installed, and are familiar
with the basics of tensor operations.

### MNIST data setup

We will use the classic [MNIST](http://deeplearning.net/data/mnist/) dataset,
which consists of black-and-white images of hand-drawn digits (between 0 and 9). We download the dataset using [requests](http://docs.python-requests.org/en/master/). We will only import modules when we use them, so you can see exactly what's being used at each point.

Download dataset and store it in `./mnist` dir:

In [ ]:
import os
import requests

mnist_path = "./mnist"

os.makedirs(mnist_path, exist_ok=True)

mnist_url = "http://deeplearning.net/data/mnist/"
mnist_filename = "mnist.pkl.gz"
if not os.path.exists(os.path.join(mnist_path, mnist_filename)):
    content = requests.get(mnist_url + mnist_filename).content
    with open(os.path.join(mnist_path, mnist_filename), 'wb') as fout:
        fout.write(content)

This dataset is in numpy array format, and has been stored using pickle,
a python-specific format for serializing data.



In [ ]:
import pickle
import gzip

with gzip.open(os.path.join(mnist_path, mnist_filename), "rb") as fin:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(fin, encoding="latin-1")

Each image is 28 x 28, and is being stored as a flattened row of length
784 (=28x28). Let's take a look at one; we need to reshape it to 2d
first.



In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

plt.imshow(x_train[0].reshape((28, 28)), cmap="gray")
print(x_train.shape)

PyTorch uses ``torch.tensor``, rather than numpy arrays, so we need to
convert our data.



In [ ]:
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
n, c = x_train.shape

print("x_train.shape =", x_train.shape)
print("y_train.shape =", y_train.shape)
print("y_train.min(), y_train.max() =", y_train.min(), y_train.max())

### Neural net from scratch (no torch.nn)

Let's first create a model using nothing but PyTorch tensor operations. We're assuming
you're already familiar with the basics of neural networks.

PyTorch provides methods to create random or zero-filled tensors, which we will
use to create our weights and bias for a simple linear model. These are just regular
tensors, with one very special addition: we tell PyTorch that they require a
gradient. This causes PyTorch to record all of the operations done on the tensor,
so that it can calculate the gradient during back-propagation *automatically*!

For the weights, we set ``requires_grad`` **after** the initialization, since we
don't want that step included in the gradient. (Note that a trailling ``_`` in
PyTorch signifies that the operation is performed in-place.)

**Note:** we are initializing the weights here with [Xavier initialisation](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) (by multiplying with $\frac{1}{\sqrt{n}}$).

In [ ]:
weights = torch.randn(784, 10) / np.sqrt(784)
weights.requires_grad_()

bias = torch.zeros(10, requires_grad=True)

Thanks to PyTorch's ability to calculate gradients automatically, we can
use any standard Python function (or callable object) as a model! So
let's just write a plain matrix multiplication and broadcasted addition
to create a simple linear model. We also need an activation function, so
we'll write `LogSoftmax` and use it. `LogSoftmax` is the same as `Softmax`, but **logarithmazied**:

$$
Softmax(z)_i = \sigma (z)_i= \frac{e^{x_i}}{\sum_i{e^{x_i}}}
$$

$$
LogSoftmax(z)_i = \log{\left(Softmax(z)_i\right)}
$$

Final prediction of the classifier is the index returned by `argmax` along the output vector. It's not affected by `log` function, because `log` is a monotonically increasing function and it doesn't change the order relationship of vector's values. So using `LogSoftmax` instead of `Softmax` doesn't spoil anything and visa-versa gives us 2 nice features:
1. To calculate negative log-likelihood loss we'll have to `log` our predictions anyway => it's **faster**, because there is no duplicating of operations
2. Calculating `exp` can be numerically unstable => `log` gives us more **stability**

#### Task 1 (1 point). Basic logisting regression with  `LogSoftmax`.

Expand `LogSoftmax` equation:
$$
LogSoftmax(z)_i = \log{\left(Softmax(z)_i\right)} = \#\#~\text{your}~\LaTeX~\text{here}
$$

Implement `LogSoftmax` (use expanded version, that you got above):

In [ ]:
def log_softmax(x):
    result = ## your code here
    return result

And now our model is defined like this:

In [ ]:
def model(xb):
    return log_softmax(xb.mm(weights) + bias)

We will call our function on one batch of data (in this case, 64 images). This is
one *forward pass*. Note that our predictions won't be any better than random at this stage, since we start with random weights.

In [ ]:
bs = 64  # batch size

xb = x_train[0:bs]  # a mini-batch from x
preds = model(xb)  # predictions
preds[0], preds.shape
print(preds[0], preds.shape)

As you see, the ``preds`` tensor contains not only the tensor values, but also a
gradient function. We'll use this later to do backprop.

As a loss function we'll use **negative log-likelihood loss** (or log-loss):
$$
nll(y^{pred}, y) = -\frac{1}{N} \sum_{i=0}^{N-1} \sum_{c=0}^{C-1} y_{ic}\log{y_{ic}^{pred}}
$$

where $N$ - batch size, $C$ - number of classes, $y_i$ - ground truth one-hot vector.

Let's implement **nll**! But, remember 2 things:
1. (!) Predictions of our model are already logarithmazied
2. There is only one non-zero element in $y_i$, so `nll` can be implemented very efficiently

In [ ]:
def nll(input, target):
    result = ## your code here
    return result

loss_func = nll

Let's check our loss with our random model, so we can see if we improve
after a backprop pass later.

In [ ]:
yb = y_train[0:bs]
print(loss_func(preds, yb))

Let's also implement a function to calculate the accuracy of our model.
For each prediction, if the index with the largest value matches the
target value, then the prediction was correct.

In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    result = ## your code here
    return result

Let's check the accuracy of our random model, so we can see if our
accuracy improves as our loss improves.

In [ ]:
print(accuracy(preds, yb))

We can now run a training loop.  For each iteration, we will:

- select a mini-batch of data (of size ``bs``)
- use the model to make predictions
- calculate the loss
- ``loss.backward()`` updates the gradients of the model, in this case, ``weights``
  and ``bias``.

We now use these gradients to update the weights and bias.  We do this
within the ``torch.no_grad()`` context manager, because we do not want these
actions to be recorded for our next calculation of the gradient.  You can read
more about how PyTorch's Autograd records operations
[here](https://pytorch.org/docs/stable/notes/autograd.html>).

**Note:** in the previous seminar we hacked the internals of `torch.tensor` not to record operations in PyTorch's Autograd (we used  `tensor.data` attribute). Above is presented a cleaner way to make gradient step.

We then set the gradients to zero, so that we are ready for the next loop. Otherwise, our gradients would record a running tally of all the operations that had happened (i.e. ``loss.backward()`` *adds* the gradients to whatever is already stored, rather than replacing them).

In [ ]:
lr = 0.5  # learning rate
epochs = 2  # how many epochs to train for

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

That's it: we've created and trained a minimal neural network (in this case, a
logistic regression, since we have no hidden layers) entirely from scratch!

Let's check the loss and accuracy and compare those to what we got
earlier. We expect that the loss will have decreased and accuracy to
have increased, and they have.



In [ ]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

Wow! Accuracy is close to 1.0. Is it overfitting?

### Using torch.nn.functional

We will now refactor our code, so that it does the same thing as before, only
we'll start taking advantage of PyTorch's ``nn`` classes to make it more concise
and flexible. At each step from here, we should be making our code one or more
of: shorter, more understandable, and/or more flexible.

The first and easiest step is to make our code shorter by replacing our
hand-written activation and loss functions with those from ``torch.nn.functional``
(which is generally imported into the namespace ``F`` by convention). This module
contains all the functions in the ``torch.nn`` library (whereas other parts of the
library contain classes). As well as a wide range of loss and activation
functions, you'll also find here some convenient functions for creating neural
nets, such as pooling functions. (There are also functions for doing convolutions,
linear layers, etc, but as we'll see, these are usually better handled using
other parts of the library.)

**Note:** if you're using negative log likelihood loss and log softmax activation,
then Pytorch provides a single function ``F.cross_entropy`` that combines
the two. So we can even remove the activation function from our model.

In [ ]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb.mm(weights) + bias

Note that we no longer call ``log_softmax`` in the ``model`` function. Let's
confirm that our loss and accuracy are the same as before:



In [ ]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

Here we got the same value for loss and accuracy just as above.

Refactor using nn.Module
-----------------------------
Next up, we'll use ``nn.Module`` and ``nn.Parameter``, for a clearer and more
concise training loop. We subclass ``nn.Module`` (which itself is a class and
able to keep track of state).  In this case, we want to create a class that
holds our weights, bias, and method for the forward step.  ``nn.Module`` has a
number of attributes and methods (such as ``.parameters()`` and ``.zero_grad()``)
which we will be using.

**Note:**``nn.Module`` (uppercase M) is a PyTorch specific concept, and is a class we'll be using a lot. ``nn.Module`` is not to be confused with the Python concept of a (lowercase ``m``) [module](https://docs.python.org/3/tutorial/modules.html>), which is a file of Python code that can be imported.

In [ ]:
from torch import nn

class MnistLogistic(nn.Module):
    def __init__(self):
        super().__init__()  # init super class
        
        self.weights = nn.Parameter(torch.randn(784, 10) / np.sqrt(784))  # nn.Parameter has requires_grad=True
        self.bias = nn.Parameter(torch.zeros(10))

    def forward(self, xb):
        return xb.mm(self.weights) + self.bias

Since we're now using an object instead of just using a function, we
first have to instantiate our model:



In [ ]:
model = MnistLogistic()

Now we can calculate the loss in the same way as before. Note that
``nn.Module`` objects are used as if they are functions (i.e they are
*callable*), but behind the scenes Pytorch will call our ``forward``
method automatically.



In [ ]:
print(loss_func(model(xb), yb))

Previously for our training loop we had to update the values for each parameter
by name, and manually zero out the grads for each parameter separately, like this:
```python
  with torch.no_grad():
      weights -= weights.grad * lr
      bias -= bias.grad * lr
      weights.grad.zero_()
      bias.grad.zero_()
```

Now we can take advantage of model.parameters() and model.zero_grad() (which
are both defined by PyTorch for ``nn.Module``) to make those steps more concise
and less prone to the error of forgetting some of our parameters, particularly
if we had a more complicated model:
```python
  with torch.no_grad():
      for p in model.parameters():
          p -= p.grad * lr
      model.zero_grad()
```

#### Task 2 (1 point). Implement train-loop in nn.Module style (as shown above).

In [ ]:
## your code here

Let's double-check that our loss has gone down:



In [ ]:
print(loss_func(model(xb), yb))

### Refactor using nn.Linear

We continue to refactor our code. Instead of manually defining and initializing ``self.weights`` and ``self.bias``, and calculating ``xb.mm(self.weights) + self.bias``, we will instead use the Pytorch class [nn.Linear](https://pytorch.org/docs/stable/nn.html#linear-layers) for a linear layer, which does all that for us. Pytorch has many types of predefined layers that can greatly simplify our code, and often makes it faster too.

In [ ]:
class MnistLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)

    def forward(self, xb):
        return self.lin(xb)

We instantiate our model and calculate the loss in the same way as before:



In [ ]:
model = MnistLogistic()
print(loss_func(model(xb), yb))

In [ ]:
lr = 0.5
epochs = 2

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        with torch.no_grad():
            for p in model.parameters():
                p -= p.grad * lr
            model.zero_grad()

print(loss_func(model(xb), yb))

### Refactor using optim

Pytorch also has a package with various optimization algorithms, ``torch.optim``.
We can use the ``step`` method from our optimizer to take a forward step, instead
of manually updating each parameter.

This will let us replace our previous manually coded optimization step:
```python
  with torch.no_grad():
      for p in model.parameters(): p -= p.grad * lr
      model.zero_grad()
```

and instead use just:
```
  opt.step()
  opt.zero_grad()
```
(``optim.zero_grad()`` resets the gradient to 0 and we need to call it before
computing the gradient for the next minibatch.)



In [ ]:
from torch import optim

#### Task 3 (1 point). Optimize model using [optim.SGD](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD).

In [ ]:
lr = 0.5
epochs = 2

model = MnistLogistic()
opt = ## your code here
print(loss_func(model(xb), yb))

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        # optimization step
        ## your code here

print(loss_func(model(xb), yb))

### Refactor using Dataset


PyTorch has an abstract Dataset class.  A Dataset can be anything that has a ``__len__`` function (called by Python's standard ``len`` function) and a ``__getitem__`` function as a way of indexing into it. [This tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) walks through a nice example of creating a custom ``FacialLandmarkDataset`` class as a subclass of ``Dataset``.

PyTorch's [TensorDataset](https://pytorch.org/docs/stable/_modules/torch/utils/data/dataset.html#TensorDataset>) is a Dataset wrapping tensors. By defining a length and way of indexing, this also gives us a way to iterate, index, and slice along the first dimension of a tensor. This will make it easier to access both the independent and dependent variables in the same line as we train.

In [ ]:
from torch.utils.data import TensorDataset

Both ``x_train`` and ``y_train`` can be combined in a single ``TensorDataset``,
which will be easier to iterate over and slice.



In [ ]:
train_ds = TensorDataset(x_train, y_train)

Previously, we had to iterate through minibatches of x and y values separately:
::
    xb = x_train[start_i:end_i]
    yb = y_train[start_i:end_i]


Now, we can do these two steps together:
::
    xb,yb = train_ds[i*bs : i*bs+bs]




In [ ]:
lr = 0.5
epochs = 2

model = MnistLogistic()
opt = optim.SGD(model.parameters(), lr=lr)

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        xb, yb = train_ds[i * bs: i * bs + bs]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

### Refactor using DataLoader

Pytorch's ``DataLoader`` is responsible for managing batches. You can create a ``DataLoader`` from any ``Dataset``. ``DataLoader`` makes it easier to iterate over batches. Rather than having to use ``train_ds[i*bs : i*bs+bs]``, the DataLoader gives us each minibatch automatically.

In [ ]:
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)

Previously, our loop iterated over batches (xb, yb) like this:
```python
      for i in range((n - 1) // bs + 1):
          xb, yb = train_ds[i*bs : i*bs+bs]
          pred = model(xb)
```       
 

Now, our loop is much cleaner, as (xb, yb) are loaded automatically from the data loader:
```python
      for xb, yb in train_dl:
          pred = model(xb)
```

In [ ]:
lr = 0.5
epochs = 2

model = MnistLogistic()
opt = optim.SGD(model.parameters(), lr=lr)

for epoch in range(epochs):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

Thanks to Pytorch's ``nn.Module``, ``nn.Parameter``, ``Dataset``, and ``DataLoader``,
our training loop is now dramatically smaller and easier to understand. Let's
now try to add the basic features necessary to create effecive models in practice.

### Add validation

In section 1, we were just trying to get a reasonable training loop set up for use on our training data.  In reality, you **always** should also have a validation set, in order to identify if you are overfitting.

Shuffling the training data is [important](https://www.quora.com/Does-the-order-of-training-data-matter-when-training-neural-networks>) to prevent correlation between batches and overfitting. On the other hand, the validation loss will be identical whether we shuffle the validation set or not. Since shuffling takes extra time, it makes no sense to shuffle the validation data.

We'll use a batch size for the validation set that is twice as large as that for the training set. This is because the validation set does not need backpropagation and thus takes less memory (it doesn't need to store the gradients). We take advantage of this to use a larger batch size and compute the loss more quickly.

#### Task 4 (1 point). Setup validation using recommendations given above.

In [ ]:
train_ds = ## your code here
train_dl = ## your code here

valid_ds = ## your code here
valid_dl = ## your code here

We will calculate and print the validation loss at the end of each epoch.

(Note that we always call ``model.train()`` before training, and ``model.eval()``
before inference, because these are used by layers such as ``nn.BatchNorm2d``
and ``nn.Dropout`` to ensure appropriate behaviour for these different phases.)



In [ ]:
lr = 0.5
epochs = 2

model = MnistLogistic()
opt = optim.SGD(model.parameters(), lr=lr)

for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    # calculate tolal validation loss
    ## your code here

    print(epoch, valid_loss / len(valid_dl))

### nn.Sequential

``torch.nn`` has another handy class we can use to simply our code:
[Sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential>).
A ``Sequential`` object runs each of the modules contained within it, in a
sequential manner. This is a simpler way of writing our neural network.

Let's create 2-layer (OMG) neural network with `ReLU` nonlinearity:

In [ ]:
model = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

In [ ]:
lr = 0.5
epochs = 2

opt = optim.SGD(model.parameters(), lr=lr)

for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)

    print(epoch, valid_loss / len(valid_dl))

Using your GPU
---------------

If you're lucky enough to have access to a CUDA-capable GPU (you can
rent one for about about $0.50/hour from most cloud providers) you can
use it to speed up your code. First check that your GPU is working in
Pytorch:



In [ ]:
print(torch.cuda.is_available())

And then create a device object for it:



In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Let's update ``preprocess`` to move batches to the GPU:



In [ ]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

Finally, we can move our model to the GPU.



In [ ]:
lr = 0.5
epochs = 2

model = MnistLogistic()
model.to(device)

opt = optim.SGD(model.parameters(), lr=lr)

You should find it runs faster now:



In [ ]:
for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        xb = xb.to(device)
        yb = yb.to(device)
        
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(model(xb.to(device)), yb.to(device)) for xb, yb in valid_dl)

    print(epoch, valid_loss / len(valid_dl))

Closing thoughts
-----------------

We now have a general data pipeline and training loop which you can use for
training many types of models using Pytorch.

Of course, there are many things you'll want to add, such as data augmentation,
hyperparameter tuning, monitoring training, transfer learning, and so forth, but we'll learn it later.

We promised at the start of this tutorial we'd explain through example each of
``torch.nn``, ``torch.optim``, ``Dataset``, and ``DataLoader``. So let's summarize
what we've seen:

 - **torch.nn**

   + ``Module``: creates a callable which behaves like a function, but can also
     contain state(such as neural net layer weights). It knows what ``Parameter`` (s) it
     contains and can zero all their gradients, loop through them for weight updates, etc.
   + ``Parameter``: a wrapper for a tensor that tells a ``Module`` that it has weights
     that need updating during backprop. Only tensors with the `requires_grad` attribute set are updated
   + ``functional``: a module(usually imported into the ``F`` namespace by convention)
     which contains activation functions, loss functions, etc, as well as non-stateful
     versions of layers such as convolutional and linear layers.
 - ``torch.optim``: Contains optimizers such as ``SGD``, which update the weights
   of ``Parameter`` during the backward step
 - ``Dataset``: An abstract interface of objects with a ``__len__`` and a ``__getitem__``,
   including classes provided with Pytorch such as ``TensorDataset``
 - ``DataLoader``: Takes any ``Dataset`` and creates an iterator which returns batches of data.